In [ ]:
#@title 데이터 전처리
# ({'불안':0,'당황':1,'슬픔':2,'분노':3,'상처':4,'기쁨':5})

import pandas as pd
import numpy as np

raw = pd.read_excel('/content/drive/MyDrive/Education/classifications/sentiment_train.xlsx')
raw2 = pd.read_excel('/content/drive/MyDrive/Education/classifications/sentiment_valid.xlsx')
train_data = pd.DataFrame(raw)
train_data.drop('Unnamed: 0',axis=1,inplace=True)
valid_data = pd.DataFrame(raw2)
valid_data.drop('Unnamed: 0',axis=1,inplace=True)
train_data.head()

,sentence,label
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,3
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,3
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,3
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,3
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,3


In [ ]:
# train_data[train_data['label']==5] # 기쁨이 몇개인지
print(len(train_data))
print(len(valid_data))
total_data = pd.concat([train_data, valid_data],axis=0)
total_data.reset_index(inplace=True, drop=True)
print(len(total_data))
print(total_data)


51630
6641
58271
                                                sentence  label
0      일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...      3
1      이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....      3
2      회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...      3
3      직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...      3
4      얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...      3
...                                                  ...    ...
58266  나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...      1
58267  친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아...      1
58268  남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.요...      1
58269  나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...      1
58270  친구들 모두 결혼하고 나만 혼자 남아서 쓸쓸하네.맞아. 하지만 그렇다고 아무나하고 ...      1

[58271 rows x 2 columns]


In [ ]:
total_data[total_data['label']==5]
# 긍/부정 비율 3:7

In [ ]:
#@title 필요한 라이브러리 설치

#필요 패키지 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
#KoBERT 깃허브에서 불러오기
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
# !pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

In [ ]:
torch.__version__

'1.7.1'

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
#transformer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
len(total_data)

58271

In [ ]:
device = torch.device("cuda:0")

In [ ]:
#@title data 를 list로

data_list = total_data.values.tolist()
data_list

In [ ]:
#@title 데이터셋 분리

from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train)) # 38722 출력
print(len(dataset_test)) # 12908 출력

43703
14568


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 70 # 64->70
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  assert self._num_workers == 0


In [ ]:
#@title 모델 정의 및 학습

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-53-480b6a139979>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/683 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8662846088409424 train acc 0.1875
epoch 1 batch id 201 loss 1.3156465291976929 train acc 0.3039490049751244
epoch 1 batch id 401 loss 0.8749586939811707 train acc 0.44724127182044887
epoch 1 batch id 601 loss 0.8901268839836121 train acc 0.5055116472545758
epoch 1 train acc 0.5221615865832556


<ipython-input-53-480b6a139979>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/228 [00:00<?, ?it/s]

...

epoch 4 test acc 0.67109375


  0%|          | 0/683 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.5505626201629639 train acc 0.8125
epoch 5 batch id 201 loss 0.491952121257782 train acc 0.800839552238806
epoch 5 batch id 401 loss 0.23956039547920227 train acc 0.8147599750623441
epoch 5 batch id 601 loss 0.4688952565193176 train acc 0.8183756239600666
epoch 5 train acc 0.8190777652069746


  0%|          | 0/228 [00:00<?, ?it/s]

epoch 5 test acc 0.6717927631578947


In [ ]:
#@title 모델 저장

PATH = '/content/drive/MyDrive/' # google 드라이브 연동 해야함. 관련코드는 뺐음
torch.save(model, PATH + 'totaldata_model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict2.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [ ]:
import torch 

saved_model = torch.load('/content/drive/MyDrive/totaldata_model.pt')
saved_model

In [ ]:
#@title 결과물 테스트

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    saved_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = saved_model(token_ids, valid_length, segment_ids)
# {'불안':0,'당황':1,'슬픔':2,'분노':3,'상처':4,'기쁨':5})
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불안이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 3:
                test_eval.append("분노가")
            elif np.argmax(logits) == 4:
                test_eval.append("상처가")
            elif np.argmax(logits) == 5:
                test_eval.append("기쁨이")

        print(test_eval[0] + " 느껴집니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
end = 1
while end == 1 :
    sentence = input("일기 문장 입력 : ")
    if sentence == "" :
        break
    predict(sentence)
    print("\n")

일기 문장 입력 : 봄날씨가 찾아왔다 꽃향기를 맡으니 기분이 좋다.
기쁨이 느껴집니다.


일기 문장 입력 : 친구 장례식장에 갔다.
당황이 느껴집니다.


일기 문장 입력 : 친구 생일파티에 갔다.
기쁨이 느껴집니다.


일기 문장 입력 : 어제까지 날씨가 무척 추웠는데 오늘은 봄이 되었다, 그래서 산책을 했는데 기분이 좋았다., 그런데  똥을 밟았다.
기쁨이 느껴집니다.


일기 문장 입력 : 어제까지 날씨가 무척 추웠는데 오늘은 봄이 되었다, 그래서 산책을 했는데 기분이 좋았다. 그러나 똥을 밟았다.
슬픔이 느껴집니다.


일기 문장 입력 : 어제까지 날씨가 무척 추웠는데 오늘은 봄이 되었다, 그래서 산책을 했는데 기분이 좋았다 그런데 똥을 밟았다.
슬픔이 느껴집니다.


일기 문장 입력 : 오늘 하루종일 기분이 안좋았는데 잘생긴 얼굴을 보니 기분이 좋아졌다.
기쁨이 느껴집니다.


일기 문장 입력 : 어제 하루종일 날씨가 꿀꿀하더니 오늘도 꿀꿀하네
기쁨이 느껴집니다.


일기 문장 입력 : 어제 하루종일 날씨가 꿀꿀하니 기분이 안좋았다.
슬픔이 느껴집니다.


일기 문장 입력 : 친구가 내 발을 밟아서 당황스러웠다.
당황이 느껴집니다.


일기 문장 입력 : 친구 장례식장에 갔다.
당황이 느껴집니다.


일기 문장 입력 : 할머니가 맛있는 밥을 해주셔서 엄청 배부르게 먹었다.
기쁨이 느껴집니다.


일기 문장 입력 : 배는 아픈데 똥이 안나온다. 변비에 걸린걸까
불안이 느껴집니다.


일기 문장 입력 : 운동을 열심히 했더니 배에 복근이 생길 것 같다.
기쁨이 느껴집니다.


일기 문장 입력 : 요즘 운동을 못했더니 살이 쪘다.
당황이 느껴집니다.


일기 문장 입력 : 
